## Problem Statement:

### In this project, we have to take aligned images from CelebA dataset and detect whether they have specific attributes.
### The attributes we are interested in are: (male/female; smile/not; mustache-beard/not; bangs/not; eyeglasses/not; hat/not; blonde/not; pale skin; attractive/not; blurry/not).

### This is basically a multi-label classification problem which aims to target multiple attributes.
### Therefore, we will develop a model which have an output layer with 10 neurons.

In [0]:
# Install a drive fuse wrapper
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

# Create a directory "drive" and mount Google Drive from it
!mkdir -p drive
#!ls drive/
!google-drive-ocamlfuse drive

In [0]:
print('My Google Drive/Datasets Files:') # To test whether the drive folder is mounted in Colab.
!ls drive/Datasets


In [0]:
# Import needed libraries to run CNN
from PIL import Image
import numpy as np

import keras
from keras import applications
from keras.applications import VGG16 
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model 
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras.optimizers import SGD
from keras import backend as k 
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
from keras.preprocessing import image

import random

Using TensorFlow backend.


## Dataset Reference: http://mmlab.ie.cuhk.edu.hk/projects/CelebA.html

####S. Yang, P. Luo, C. C. Loy, and X. Tang, "From Facial Parts Responses to Face Detection: A Deep Learning Approach", in IEEE International Conference on Computer Vision (ICCV), 2015

In [0]:
# UNZIP YOUR DATA FROM GOOGLE DRIVE TO GOOGLE COLAB AS IT IS FASTER TO READ IMAGES FROM GOOGLE COLAB DIRECTLY
# This step will take maximum 2 mins
import zipfile
zip_ref = zipfile.ZipFile('drive/Datasets/CelebA/Img/img_align_celeba.zip', 'r')
zip_ref.extractall()
zip_ref.close()

In [0]:
# parameters
ImgSz = 224

## Create function that takes
# f: list contains your dataset in this format [image_path /space/ label]
# b_start, b_end: starting and ending points of the batch that you want to be extracted
# OUTPUT:
# images and labels of size (b_end - b_start + 1)

def LoadData(f, b_start, b_end):
  
  random.shuffle(f)
  b_end = min(len(f), b_end)
  
  labels = []
  images = []
  for i in range(b_start, b_end):
    dt = f[i].split(" ")
    #Reading Image
    im = image.load_img(dt[0])  
    im = im.resize((ImgSz, ImgSz))
    x = image.img_to_array(im) 
    #x = x.reshape((1,) + x.shape) 
    x = np.array(x, dtype="float") / 255.0

    images.append(x)
    labels.append([int(lx) for lx in dt[1:]])

  lbls = np.array(labels)
  imgs = np.array(images)

  return imgs, lbls


In [0]:
# Function to be given to fit_generator. This function is responsible of 
# providing fit_generator with batch of images and their labels
# INPUT:
# f: list contains your dataset in this format [image_path /space/ label]
# batch_size: number of images per batch (CNN iteration)
def imageLoader(f, batch_size):

    numOfImgs = len(f)

    # make the generator infinite  
    while True:
      
      batch_start = 0
      batch_end = batch_size
      #random.shuffle(f)
      while batch_start < numOfImgs:
          [X, Y] = LoadData(f, batch_start, batch_end)
          yield (X,Y) #a tuple with two numpy arrays with batch_size samples     

          batch_start += batch_size   
          batch_end += batch_size

## Our Model

## In this project, we developed a multilabel classifier using pre-trained CNNs.
### As a pre-trained model we used VGG16 which is trained using imagenet data.
#### Reference: https://github.com/fchollet/deep-learning-models/releases/download/v0.1/vgg16_weights_tf_dim_ordering_tf_kernels.h5

## Model Structure:
### In this model, we trained the last hidden layer of VGG16 from scratch.
### Therefore, we removed one hidden layer + output layer.

### The layers we added:
#### --Dense layer with 4096 units -> The last hidden layer we trained from scratch.
#### --Dropout layer with rate 0.5 -> To prevent overfitting.
#### --Dense layer with 10 units and sigmoid activation function. -> 10 attributes, multilabel classification

#### Our problem is a multilabel classification problem because we assign multiple targets (attributes) to each image!
#### Therefore, sigmoid activation function is used instead of softmax activation function.

In [0]:
# All layers will be freezed except last 2 hidden layers will be trained from scratch

#Loading VGG model
VGGmodel = VGG16(weights = "imagenet")
VGGmodel.summary() # Print the network

# Freeze the weight of the rest of the layers
for layer in VGGmodel.layers[:-4]: # We want to train the last 2 layers of VGG.
    layer.trainable = False
    
# Remove last two layers (one hidden layers + Output layer)
VGGmodel.layers.pop()
VGGmodel.layers.pop()

# Check the status of the layers 
for layer in VGGmodel.layers:
    print(layer, layer.trainable)
    
# Create the model
model = Sequential()
 
# Add the vgg convolutional base model
model.add(VGGmodel)
 
# Add new layers
model.add(Dense(4096, activation='relu')) # Put a dense layer instead of the hidden layer
model.add(Dropout(0.5)) # Put a dropout layer to prevent the overfitting
model.add(Dense(10, activation='sigmoid')) # Output layer


 
# Show a summary of the model. Check the number of trainable parameters
model.summary()
# Check the status of the layers 
for layer in model.layers:
    print(layer, layer.trainable)

553467904/553467096 [==============================] - 36s 0us/step
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_p

In [0]:
!ls img_align_celeba | grep "000001.jpg" # To test whether the images are extracted successfully.

000001.jpg


## The code to create train, validation and test sets.

### 1. Partition file is read to get which image belongs to which set (train, validation or test).
### 2. Attribute file is read to get the labels of each image
### 3. Attibutes which have the values "-1" are converted to 0 because we use binary_crossentropy.
### 4. The attributes we are interested in are taken together with the corresponding image.
### 5. Training set is being taken by looking at the 0 values in the partition file.
### 6. Training set is written into the text file: "drive/Datasets/TRAIN_data.txt"
### 7. Validation set is being taken by looking at the values "1" in the partition file.
### 8. Validation set is written into the text file: "drive/Datasets/VALIDATION_data.txt"
### 9. Test set is being taken by looking at the values "2" in the partition file.
### 10. Test set is written into the text file: "drive/Datasets/TEST_data.txt"

In [0]:
while True: # In the first run, Colab may not be able to read the data.
  
  try:
    
    eval_file = open('drive/Datasets/CelebA/Eval/list_eval_partition.txt', 'r').read().splitlines() # Read the partition file
    eval_file = list(filter(None, eval_file)) # remove any empty cell in the list

    labels_file = open('drive/Datasets/CelebA/Anno/list_attr_celeba.txt', 'r').read().splitlines() # Read the attribute file
    labels_file = list(filter(None, labels_file)) # remove any empty cell in the list
    labels_file = labels_file[2:]

    #Attribute Labels: 20, 31, 22, 24, 5, 15, 35, 9, 26, 2, 10
    labels_file = [line.split() for line in labels_file]


    for index, line in enumerate(labels_file): # -1s are converted to 0s in the attributes
      for index2, label in enumerate(line):
        if label == "-1":
          labels_file[index][index2] = "0"
    

    # Interested attributes are taken
    labels_file = [line[0] + " " + line[21] + " " + line[32] + " " + str(int(line[23]) or int(not(int(line[25])))) + " " + line[6] + " " + line[16]
                   + " " + line[36] + " " + line[10] + " " + line[27] + " " + line[3] + " " + line[11] for line in labels_file]

    

    # Train set is taken
    eval_file_train = [eline for eline in eval_file if eline[-1] == '0']
    eval_file_train = [eline.split()[0] for eline in eval_file_train]

    
    # Training data is written into the text file.
    with open('drive/Datasets/TRAIN_data.txt', 'w') as f:
        for item in labels_file[:int(eval_file_train[-1].split(".")[0])]:
            f.write("%s\n" % ("img_align_celeba/" + item))

    # Validation set is taken
    eval_file_validation = [eline for eline in eval_file if eline[-1] == '1']
    eval_file_validation = [eline.split()[0] for eline in eval_file_validation]

    # Validation data is written into the text file.
    with open('drive/Datasets/VALIDATION_data.txt', 'w') as f:
        for item in labels_file[int(eval_file_validation[0].split(".")[0])-1:int(eval_file_validation[-1].split(".")[0])]:
            f.write("%s\n" % ("img_align_celeba/" + item))

    # Test set is taken
    eval_file_test = [eline for eline in eval_file if eline[-1] == '2']
    eval_file_test = [eline.split()[0] for eline in eval_file_test]

    # Test set is written into the text file
    with open('drive/Datasets/TEST_data.txt', 'w') as f:
        for item in labels_file[int(eval_file_test[0].split(".")[0])-1:]:
            f.write("%s\n" % ("img_align_celeba/" + item))

            
    # Read text file contains training data in a list
    f_train = open('drive/Datasets/TRAIN_data.txt', 'r').read().splitlines()
    f_train = list(filter(None, f_train)) # remove any empty cell in the list

    # Read text file contains testing data in a list
    f_validation = open('drive/Datasets/VALIDATION_data.txt', 'r').read().splitlines()
    f_validation = list(filter(None, f_validation)) # remove any empty cell in the list

    # Read text file contains testing data in a list
    f_test = open('drive/Datasets/TEST_data.txt', 'r').read().splitlines()
    f_test = list(filter(None, f_test)) # remove any empty cell in the list
    
    if(f_train[0][:3] == "img" and f_validation[0][:3] == "img" and f_test[0][:3] == "img"):
      break
    
  except:
    
    continue

In [0]:
f_train[:5] # To see that the train set is read successfully.

['img_align_celeba/000001.jpg 0 1 0 0 0 0 0 0 1 0',
 'img_align_celeba/000002.jpg 0 1 0 0 0 0 0 0 0 0',
 'img_align_celeba/000003.jpg 1 0 0 0 0 0 0 0 0 1',
 'img_align_celeba/000004.jpg 0 0 0 0 0 0 0 0 1 0',
 'img_align_celeba/000005.jpg 0 0 0 0 0 0 0 0 1 0']

In [0]:
f_validation[:5] # To see that the validation set is read successfully.

['img_align_celeba/162771.jpg 0 1 0 1 0 0 0 0 1 0',
 'img_align_celeba/162772.jpg 1 1 1 0 0 0 0 0 0 0',
 'img_align_celeba/162773.jpg 0 0 0 0 0 0 0 0 1 0',
 'img_align_celeba/162774.jpg 1 1 1 0 0 0 0 0 1 0',
 'img_align_celeba/162775.jpg 0 0 0 0 0 0 0 0 0 0']

In [0]:
f_test[:5] # # To see that the test set is read successfully.

['img_align_celeba/182638.jpg 0 1 0 0 0 1 0 0 0 0',
 'img_align_celeba/182639.jpg 0 0 0 0 0 0 0 0 0 0',
 'img_align_celeba/182640.jpg 0 1 0 0 0 0 0 0 1 0',
 'img_align_celeba/182641.jpg 0 1 0 0 0 0 0 0 1 0',
 'img_align_celeba/182642.jpg 0 1 0 0 0 0 0 0 0 0']

## Model Compilation

### Epoch = 9
#### -- Even though we used earlystopping technique, we wanted to train at most 9 epochs because Google Colab's GPU was -- running out of memory and it did not allow us to train the model more previously.

### Optimizer = SGD with learning rate 0.1 because we also tried Adam but SGD gave a better result in general.
### Loss function = binary_crossentropy because we developed a multilabel classifier which targets multiple labels at the same time.
#### Metrics = Binary accuracy is used to get the general training accuracy for each target.

### **In our model, earlystopping technique is utilized based on the validation loss to prevent the overfitting and have better generalized model.

### **Therefore, we did not need to check the validation data after the model is trained. We take the model which generalizes the best on the validation data.

### **We do not train our model on the test data! Validation set is used to see the generalization!

### Notice: Since our model generalized better at each epoch more, it could reached until epoch 8 without stopping. Maybe it would be better to train more but Google Colab did not allow us to do that because of memory constraints and stopped training at the 8th epoch.

In [0]:
epochs = 9
batch_size = 20

# Compile the model
model.compile(optimizer=SGD(0.1),loss="binary_crossentropy", metrics=["accuracy"])

earlystopper = EarlyStopping(monitor='val_loss', min_delta=0, patience=2, verbose=0, mode='auto', baseline=None, restore_best_weights=True)

# TRAINING
model.fit_generator(imageLoader(f_train, batch_size), steps_per_epoch=len(f_train)/batch_size, nb_epoch = epochs, 
                    verbose=1, validation_data=imageLoader(f_validation, batch_size), validation_steps=len(f_validation)/batch_size, callbacks=[earlystopper])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., steps_per_epoch=8138.5, verbose=1, validation_data=<generator..., validation_steps=993.35, callbacks=[<keras.ca..., epochs=9)`
  # This is added back by InteractiveShellApp.init_path()


Epoch 1/9
8139/8138 [==============================] - 2883s 354ms/step - loss: 0.3726 - acc: 0.8275 - val_loss: 0.3164 - val_acc: 0.8577
Epoch 2/9
8139/8138 [==============================] - 2893s 355ms/step - loss: 0.3058 - acc: 0.8657 - val_loss: 0.2986 - val_acc: 0.8686
Epoch 3/9
8139/8138 [==============================] - 2913s 358ms/step - loss: 0.2810 - acc: 0.8833 - val_loss: 0.2706 - val_acc: 0.8849
Epoch 4/9
8139/8138 [==============================] - 2916s 358ms/step - loss: 0.2592 - acc: 0.8928 - val_loss: 0.2488 - val_acc: 0.8936
Epoch 5/9
8139/8138 [==============================] - 2924s 359ms/step - loss: 0.2476 - acc: 0.8967 - val_loss: 0.2431 - val_acc: 0.8984
Epoch 6/9
8139/8138 [==============================] - 2913s 358ms/step - loss: 0.2390 - acc: 0.9013 - val_loss: 0.2359 - val_acc: 0.9032
Epoch 7/9
8139/8138 [==============================] - 2915s 358ms/step - loss: 0.2324 - acc: 0.9051 - val_loss: 0.2254 - val_acc: 0.9082
Epoch 8/9
1823/8138 [=====>.......

## Testing on the test data to see the binary accuracy:

In [0]:
scores = model.evaluate_generator(imageLoader(f_test, batch_size), steps = len(f_test)/batch_size)
print('Final test accuracy:', (scores[1]*100.0))


Final test accuracy: 89.805128430809


## The functions below are written to get the accuracy for each individual attribute.

### 1. getTargetPred function takes a line (data for an img) from the test set and returns the predictions of the attributes together with its true attributes.

In [0]:
def getTargetPred(line):
  
  dt = line.split(" ")
    
  im = image.load_img(dt[0])  
  im = im.resize((ImgSz, ImgSz))
  x = image.img_to_array(im) 
  #x = x.reshape((1,) + x.shape) 
  x = np.array(x, dtype="float") / 255.0
    
  img = np.array([x])
  y_pred = model.predict(img)
  y_target = np.array([[int(lx) for lx in dt[1:]]])
  return y_target[0], y_pred[0]

### 2. getPerAcc function takes the test set file and calculates the accuracies for each individual attribute together with the average accuracy of them.

In [0]:
def getPerAcc(f):
  print("Calculating the accuracy of each attribute.")
  print("This may take about 10 minutes...")
  accs = [0] * 10
  for ind, line in enumerate(f):
    y_target, y_pred = getTargetPred(line)
    
    for index in range(len(y_pred)):
      if (y_pred[index] <= 0.5 and y_target[index] == 0) or (y_pred[index] > 0.5 and y_target[index] == 1):
        accs[index] += 1
  
  print()
  print("Accuracy of each attribute:")
  numS = len(f)
  #Attribute Labels: 20, 31, 22-24, 5, 15, 35, 9, 26, 2, 10
  attLabels = {0:"male/female", 1:"smile/not" , 2:"mustache-beard/not", 3:"bangs/not" , 
               4:"eyeglasses/not" , 5:"hat/not" , 6:"blonde/not" , 7:"pale skin", 8:"attractive/not", 9:"blurry/not"}
  
  totalAcc = 0
  for inn, cl in enumerate(accs):
    totalAcc += cl/numS
    print(attLabels[inn] + ": " + str(cl/numS))
  
  print()
  print("Average Accuracy:", totalAcc/float(len(accs)))
   
    

## Result:

### 1. Accuracy per attribute
### 2. Average accuracy

In [0]:
getPerAcc(f_test)

Calculating the accuracy of each attribute.
This may take about 10 minutes...

Accuracy of each attribute:
male/female: 0.8979060214407374
smile/not: 0.8293758140466887
mustache-beard/not: 0.8810740406772869
bangs/not: 0.9268109407874963
eyeglasses/not: 0.9466486324015629
hat/not: 0.9577697625488428
blonde/not: 0.8926460274521592
pale skin: 0.9579200480913737
attractive/not: 0.7353972547840898
blurry/not: 0.9494038673479611

Average Accuracy: 0.8974952409578197


## Summary

### Our model performed pretty well for the attributes except the attribute "attractiveness".
### Average accuracy is also so close to 90% and it is pretty well in our own opinion.

